In [ ]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter

In [ ]:
ckpt_converter = 'checkpoints_v2/converter'
device = "cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs_v2'
print(device)

In [ ]:
tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')
os.makedirs(output_dir, exist_ok=True)

In [ ]:
reference_speaker = '/media/vitaliy/Samsung_T5/terebi-languages/assets/presenters/other/Japanese_Female_TianDianLunZi.mp3'
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, vad=False)

In [ ]:
from melo.api import TTS

In [ ]:
texts = {
    # 'EN_NEWEST': "But I believe that humanity can come together and hold hands, Kokoro.",  # The newest English base speaker model
    # 'EN': "But I believe that humanity can come together and hold hands, Kokoro.",
    'JP': "結局",
}
src_path = f'{output_dir}/tmp.wav'
speed = 1.0

In [ ]:
for language, text in texts.items():
    model = TTS(language=language, device=device)
    speaker_ids = model.hps.data.spk2id
    
    for speaker_key in speaker_ids.keys():
        speaker_id = speaker_ids[speaker_key]
        speaker_key = speaker_key.lower().replace('_', '-')
        
        source_se = torch.load(f'checkpoints_v2/base_speakers/ses/{speaker_key}.pth', map_location=device)
        model.tts_to_file(text, speaker_id, src_path, speed=speed, sdp_ratio=0.5)
        save_path = f'{output_dir}/output_v2_{speaker_key}.wav'

        # Run the tone color converter
        encode_message = "@MyShell"
        tone_color_converter.convert(
            audio_src_path=src_path, 
            src_se=source_se, 
            tgt_se=target_se, 
            output_path=save_path,
            message=encode_message, tau=0.5)